In [ ]:
%pip install -qU pypdf langchain-community langchain-text-splitters

##### RAG(Retrieval-Augmented Generation)에서, 전체 문서를 그대로 LLM에 넘기면 토큰 제한을 초과할 가능성이 큽니다.
- 토큰 제한을 초과하면 모델이 입력을 잘라내거나 오류가 발생할 수 있습니다.
- 비록 초과하지 않는다 해도, 불필요하게 긴 문서는 응답 생성 시간과 비용을 늘립니다.

##### 문서를 적절한 단위(“chunk” 혹은 “paragraph”)로 나누는 이유
- 한 번에 모델에 넣을 수 있는 텍스트 분량(맥시멈 토큰 수)을 넘지 않도록 나누어 처리합니다.
- 검색(Retrieval) 시 사용자 질문과 가장 관련성 높은 문단만 골라서 LLM에 전달하므로, 토큰 사용량을 줄이고 답변 품질을 높입니다.

##### Retriever가 Paragraph(또는 Chunk) 단위로 검색
- 문서를 일정 크기로 “청크”로 나눈 뒤, 각 청크를 벡터 스토어 등에 저장합니다(임베딩).
- 질문이 들어오면 관련성이 높은 청크만 골라 LLM에 전달합니다.
- 이것이 “문단 단위로 필요한 만큼만” 모델에 주는 핵심 아이디어입니다.

##### 정리:
- 설명에 언급된 대로, 전체 문서를 통째로 보내면 토큰 초과나 비효율 문제가 생긴다.
- 이를 피하려고 문서를 적절히 나눈 뒤, 질문과 관련된 부분만 LLM에 주는 방식이 RAG에서 일반적으로 쓰이는 방법이다.
- Retriever(검색 단계)에서는 이렇게 나눈 “문단(청크)” 단위를 기반으로 검색·필터링하여 필요한 부분만 추출한다.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pdf_file_path = '../spring_framework_docs.pdf'
loader = PyPDFLoader(pdf_file_path)
pages = []
# 쪽수별로 문서를 append
async for page in loader.alazy_load():
    pages.append(page)

In [ ]:
pages

In [ ]:
# PDF 문서안에 테이블과 이미지와 같은 문서를 추출하기 위해 zerox 패키지를 사용
# https://github.com/getomni-ai/zerox
%pip install py-zerox

In [ ]:
# asyncio를 실행할 때, 이벤트루프가 없어야하나, notebook엣 default로 발생시키는 이벤트루프가 있어서, nest_asyncio를 사용하여 중첩된 이벤트루프를 사용할 수 있도록 함
%pip install -q nest_asyncio

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# 문서를 AI가 파싱하여 텍스트로 변환하는데 사용하기 떄문에 가격이 매우매우 비싸다. 조심해서 사용해야함

import os
import sys
import json
import asyncio

# 가능하면 최상단에서 호출 (주피터/파이썬 REPL 등 환경에 따라서 순서가 중요할 수 있음)
from dotenv import load_dotenv

try:
    # UTF-8 출력이 가능한 환경(일반 파이썬 실행)에서만 적용
    # 주피터/IPython 환경에서는 OutStream이어서 reconfigure 호출시 AttributeError가 날 수 있음
    sys.stdout.reconfigure(encoding='utf-8')
except AttributeError:
    pass

# .env 파일 로드 (OPENAI_API_KEY 등 환경 변수 확인)
load_dotenv()

# PyZeroX 임포트는 .env 로드 후에 하는 것을 권장
from pyzerox import zerox


def check_environment_variables() -> None:
    """
    주 사용 환경 변수(예: OPENAI_API_KEY)가 제대로 설정되어 있는지 검사.
    미설정 시 조기 종료 또는 사용자에게 경고 표시.
    """
    openai_key = os.getenv("OPENAI_API_KEY")
    if not openai_key or not openai_key.startswith("sk-"):
        raise EnvironmentError(
            "OPENAI_API_KEY가 제대로 설정되어 있지 않거나 'sk-'로 시작하지 않습니다.\n"
            "비싼 API 호출 실패를 방지하기 위해 프로그램을 종료합니다."
        )


async def process_pdf(
    file_path: str,
    model: str = "gpt-4o",
    output_dir: str = "./documents",
    select_pages=None,
    system_prompt=None,
    **kwargs
):
    """
    PDF를 처리해서 결과를 반환하는 메인 비동기 함수.
    
    :param file_path: 분석할 PDF 파일 경로
    :param model: 사용할 모델 (기본값 "gpt-4o-mini")
    :param output_dir: 처리 결과(마크다운 등) 저장 경로
    :param select_pages: 처리할 페이지 선택 (None이면 전체)
    :param system_prompt: 시스템 프롬프트
    :param kwargs: 모델별 추가 파라미터
    :return: Zerox가 생성한 결과
    """
    result = await zerox(
        file_path=file_path,
        model=model,
        output_dir=output_dir,
        custom_system_prompt=system_prompt,
        select_pages=select_pages,
        **kwargs
    )
    return result


def main():
    """
    실제 실행을 담당하는 메인 함수.
    """
    # 1) 환경 변수 점검
    check_environment_variables()

    # 2) 원하는 파라미터 설정
    file_path = "../spring_framework_docs.pdf"
    model_name = "gpt-4o"   # 필요 시 "gpt-4", "gpt-3.5-turbo" 등으로 교체
    output_dir = "./documents"
    select_pages = None  # None이면 모든 페이지 처리
    system_prompt = None # 필요 시 사용자 정의 시스템 프롬프트 지정

    # 3) 비동기 함수 실행
    try:
        result = asyncio.run(process_pdf(
            file_path=file_path,
            model=model_name,
            output_dir=output_dir,
            select_pages=select_pages,
            system_prompt=system_prompt
        ))
    except Exception as e:
        print(f"오류가 발생하여 작업을 중단합니다: {e}")
        return

    # 4) 결과 출력 (너무 길 경우 파일 저장만 하고 간단히 요약만 보여줄 수도 있음)
    print(result)


if __name__ == "__main__":
    main()


In [ ]:
%pip install -q "unstructured[md]" nltk

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,  # 1500자씩 분할
    chunk_overlap=100,  # 100자씩 중첩
    separators=['\n\n', '\n']
)

In [ ]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document

markdown_path = "./documents/spring_framework_docs.md"
loader = UnstructuredMarkdownLoader(markdown_path)
document_list = loader.load_and_split(text_splitter)

In [ ]:
document_list

In [ ]:
%pip install -q markdown html2text beautifulsoup4

In [ ]:
import markdown
from bs4 import BeautifulSoup

# 마크다운 파일 경로
markdown_path = './documents/spring_framework_docs.md'
# 변환된 텍스트를 저장할 파일 경로
text_path = './documents/spring_framework_docs.txt'

# 마크다운 파일을 읽어옵니다
with open(markdown_path, 'r', encoding='utf-8') as md_file:
    md_content = md_file.read()

# 마크다운 콘텐츠를 HTML로 변환합니다
html_content = markdown.markdown(md_content)

# HTML 콘텐츠를 파싱하여 텍스트만 추출합니다
soup = BeautifulSoup(html_content, 'html.parser')
text_content = soup.get_text()

# 추출한 텍스트를 텍스트 파일로 저장합니다
with open(text_path, 'w', encoding='utf-8') as txt_file:
    txt_file.write(text_content)

print("Markdown converted to plain text successfully!")


In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader(text_path, encoding="utf-8")
document_list = loader.load_and_split(text_splitter)

In [ ]:
%pip install -q langchain-chroma

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model='text-embedding-3-large')

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma.from_documents(
    documents=document_list,
    embedding=embeddings,
    collection_name='spring_framework_docs',
    persist_directory='./spring_framework_docs'
)

In [ ]:
retriever = vector_store.as_retriever(search_kwargs={'k': 3})

In [ ]:
query = "스프링 프레임워크란?"

In [ ]:
# 질문과 가장 유사한 문서 3개를 검색
retriever.invoke(query)

In [ ]:
from typing_extensions import List, TypedDict

class AgentState(TypedDict):
    query: str
    context: List[Document]
    answer: str

In [ ]:
from langgraph.graph import StateGraph

graph_builder = StateGraph(AgentState)

In [ ]:
# Node는 2가지가 필요
# 1. 문서를 가져오는 retrieve
# 2. 답변을 생성하는 generate

def retrieve(state: AgentState):
    query = state['query']
    docs = retriever.invoke(query)
    return {'context': docs}

In [ ]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
from langchain_openai import ChatOpenAI

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model='gpt-4o')

In [ ]:
def generate(state: AgentState):
    context = state['context']
    query = state['query']
    rag_chain = prompt | llm
    response = rag_chain.invoke({'question': query, 'context': context})
    response = llm.invoke(query, context)
    return {'answer': response.content}

In [ ]:
graph_builder.add_node('retrieve', retrieve)
graph_builder.add_node('generate', generate)

In [ ]:
from langgraph.graph import START, END

graph_builder.add_edge(START, 'retrieve')
graph_builder.add_edge('retrieve', 'generate')
graph_builder.add_edge('generate', END)

In [ ]:
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
sequence_graph_builder = StateGraph(AgentState).add_sequence([retrieve, generate])

In [ ]:
sequence_graph_builder.add_edge(START, 'retrieve')
sequence_graph_builder.add_edge('generate', END)

In [ ]:
sequence_graph = sequence_graph_builder.compile()

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
initial_state = {'query': query}
sequence_graph.invoke(initial_state)